# Graph convolutional Network

This tutorial based on [Theoretical Foundations of Graph Neural Networks](https://www.youtube.com/watch?v=uF53xsT7mjc) and [Graph Neural Networks](https://www.youtube.com/playlist?list=PLSgGvve8UweGx4_6hhrF3n4wpHf_RV76_)

### Preliminaries


##  1. Permutation Invariance Property

We require that our algorithms to satisfiy the permutation invariance property. This requirement stems from the fact that graph structured data does not involve ordering. Hence, the order of nodes should not matter for us. 



Let $X \in \mathbb{R}^{n \times d}$ represent node features and $P$ be a permutation. A function $f$ permutation invariant if

## $$ f(\textbf{P} \textbf{X}) = f(\textbf{X})$$

Let's take a look at some permutation invariant functions

## Deep Sets by Zaheer et al. NeurIPS 2017

A generic form:
## $$ f(\textbf{X}) = f(\textbf{P}\textbf{X})$$
## $$ f(\textbf{X}) = \phi \Bigg(\Sigma_i \psi(x_i) \Bigg),$$

where $\phi$ and $ \psi $ are learnable functions. $\phi$ is a __permutation invariant aggregation function__ (e.g. summation, averagining, maximization).

##  2. Permutation Equivariance Property


## $$ f(\textbf{P}\textbf{X}) = \textbf{P}f(\textbf{X})$$

## $$ f(\textbf{X}) = \phi \Bigg( \Phi_i \psi(x_i) \Bigg),$$

where $\Phi$ is a permutation-invariant aggregator (such as sum, avg or max)

In [1]:
import numpy as np
# Number of nodes and size of features
n,d=4,2

# (1) Node Representation
X=np.random.randn(n,d)

# (2) Permutation Matrix
P=np.identity(n)[np.random.permutation(n)]

print(f'X:\n{X}\n')

print(f'PX:\n{P@X}')
print('\t\t\t\t Only order is changed.')


# Sum
sumX=np.einsum("ij->",X)
sumPX=np.einsum("ij->",P@X)
print(f'Σ_i Σ_j X_ij=> {np.allclose(sumX,sumPX,atol=1e4)}, {sumX:.3f}')



# Column Sum
colsumX=np.einsum("ij-> j ",X)
colsumPX=np.einsum("ij-> j ",P@X)
print(f'Σ_i X_ij=> {np.allclose(colsumX,colsumPX,atol=1e4)}, {colsumX}')


# Row Sum
rowsumX=np.einsum("ij-> i ",X)
rowsumPX=np.einsum("ij-> i ",P@X)
print(f'Σ_j X_ij=> {np.allclose(rowsumX,rowsumPX,atol=1e4)}, {rowsumX}')

X:
[[ 0.86749028 -0.47076494]
 [ 0.72849109  0.49842976]
 [ 1.30273722 -0.50826614]
 [ 2.29124875 -0.29035236]]

PX:
[[ 1.30273722 -0.50826614]
 [ 0.86749028 -0.47076494]
 [ 2.29124875 -0.29035236]
 [ 0.72849109  0.49842976]]
				 Only order is changed.
Σ_i Σ_j X_ij=> True, 4.419
Σ_i X_ij=> True, [ 5.18996734 -0.77095369]
Σ_j X_ij=> True, [0.39672534 1.22692085 0.79447107 2.00089639]


# Learning on Graphs

We can represent edges with a __A__ be a binary adjacency matrix.

## Invariance: $$ f(\textbf{P}\textbf{X},\textbf{P} \textbf{A} \textbf{P}^T) = f(\textbf{X},\textbf{A})$$


## Equivariance: $$ f(\textbf{P}\textbf{X},\textbf{P} \textbf{A} \textbf{P}^T) = \textbf{P}f(\textbf{X},\textbf{A})$$



## Neighbourhood:
$$ \mathcal{N}_i = \{ j, (i,j) \in \mathcal{E} \vee (j,i) \in \mathcal{E} \} $$


$$ 
f(\textbf{X},\textbf{A})=\begin{bmatrix} 
- \quad g( \textbf{x}_1, \textbf{X}_{\mathcal{N}_1} ) \quad-\\
- \quad g( \textbf{x}_2, \textbf{X}_{\mathcal{N}_2} ) \quad-\\
\cdots \\
- \quad g( \textbf{x}_n, \textbf{X}_{\mathcal{N}_n} ) \quad-\\
\end{bmatrix} 
$$

g should be also permutation invariant.



# Graph Convolutional Neural Network


### $$ h_i = \phi \, \Big( x_i , \Phi_{j \in \mathcal{N}_i} c_{ij} \, \psi(x_j) \Big)$$
where features of neighbours aggregated with fixed wieght $c_{ij}$. Useful for homophilous graphs and scaling up: when edges encode label similarity

# Attention Graph Neural Network

### $$ h_i = \phi \, \big( x_i , \Phi_{j \in \mathcal{N}_i} a(x_i,x_j) \, \psi(x_j) \big)$$
where features of neighbours aggregated with implicit weights (via attention)$. Usefull: Edges need not encode homophily.

# Mesage-passing Graph Neural Network

### $$ h_i = \phi \, \big( x_i , \Phi_{j \in \mathcal{N}_i} \psi(x_i,x_j) \big)$$


# Implementation

### 1. Why does matrix product of A and X imply ?

Matrix product of an adjecency matrix $A$ with node representations $X$ "updates" each node representations $X_i$ with its neighbours $A_i$. 

Let's understand this computation in detail. Assume that $A$ is a idendity matrix, hence, the impact of neighbors of $i$th node is __zero__.

In [27]:
n,d=4,2
A=np.eye(n)
X=np.random.randn(n,d)
AX=np.einsum('ik,kj->ij', A, X)
print(f'A:\n{A}\n')
print(f'X:\n{X}\n')
print(f'AX:\n{AX}\n')

A:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]

X:
[[-0.69339122 -0.59290391]
 [ 1.16204193  0.99112958]
 [ 0.62776671 -0.20528424]
 [-1.89022329  0.11236841]]

AX:
[[-0.69339122 -0.59290391]
 [ 1.16204193  0.99112958]
 [ 0.62776671 -0.20528424]
 [-1.89022329  0.11236841]]



### 2. Why don't we extend the neighbourhood?

In [28]:
# Let's connect 
A[0][1]=1 # n_0 => n_1
A[1][0]=1 # n_1 => n_0

A[2][1]=1 # n_2 => n_1
A[1][2]=1 # n_1 => n_2

A

array([[1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [0., 1., 1., 0.],
       [0., 0., 0., 1.]])

In [29]:
# Node 0 and Node 2 are not connected
assert A[0][2]==A[2][0]==0.0

In [30]:
# Neigbours of neighbours
neighbours_of_neighbours_scalar=.5
A+= ((A+A@A ==1) *1.0) * neighbours_of_neighbours_scalar 
print(f'A:\n{A}\n')
print(A[0][2])
print(A[2][0])

A:
[[1.  1.  0.5 0. ]
 [1.  1.  1.  0. ]
 [0.5 1.  1.  0. ]
 [0.  0.  0.  1. ]]

0.5
0.5


In [31]:
AX=np.einsum('ik,kj->ij', A, X)
print(f'A:\n{A}\n')
print(f'AX:\n{AX}\n')

A:
[[1.  1.  0.5 0. ]
 [1.  1.  1.  0. ]
 [0.5 1.  1.  0. ]
 [0.  0.  0.  1. ]]

AX:
[[ 0.78253406  0.29558355]
 [ 1.09641742  0.19294143]
 [ 1.44311303  0.48939339]
 [-1.89022329  0.11236841]]



# Node Classification

In [32]:
import tensorflow as tf
import spektral

In [33]:
A, X, labels, train_mask, val_mask, test_mask = spektral.datasets.citation.load_data(dataset_name='cora')

Loading cora dataset
Pre-processing node features


In [34]:

X = X.todense()

A = A.todense()
A = A + np.eye(A.shape[0])
X = X.astype('float32')
A = A.astype('float32')

print('X:',X.shape)
print('A:',A.shape)
# Labels
print('Y:',labels.shape)

X: (2708, 1433)
A: (2708, 2708)
Y: (2708, 7)


In [36]:
print(X[0])
print(A[0])
print(labels[0])

[[0. 0. 0. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]]
[0 0 0 1 0 0 0]


In [37]:
print(train_mask[0])
print(train_mask.shape)
print(np.sum(train_mask)) # True

True
(2708,)
140


In [38]:
print(val_mask[0])
print(val_mask.shape)
print(np.sum(val_mask)) # True

False
(2708,)
500


In [39]:
print(test_mask[0])
print(test_mask.shape)
print(np.sum(test_mask)) # True

False
(2708,)
1000


In [40]:
def masked_softmax_cross_entropy(logits, labels, masks):
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
    masks = tf.cast(masks, dtype=tf.float32)
    masks /= tf.reduce_mean(masks)
    loss *= masks
    return tf.reduce_mean(loss)

In [41]:
def masked_accuracy(logits, labels, masks):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    masks = tf.cast(masks, dtype=tf.float32)
    masks /= tf.reduce_mean(masks)
    accuracy_all *= masks
    return tf.reduce_mean(accuracy_all)

In [42]:
def gnn(fts, adj, transform, activation):
    seq_fts = transform(fts)
    ret_fts = tf.matmul(adj, seq_fts)
    return activation(ret_fts)

In [43]:
def train_cora(fts, adj, gnn_fn, units=32, epochs=200, lr=.01):
    lyr_1 = tf.keras.layers.Dense(units)
    lyr_2 = tf.keras.layers.Dense(7)  # for 7 classes

    def cora_gnn(fts, adj):
        hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
        logits = gnn_fn(hidden, adj, lyr_2, tf.identity)
        return logits

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    best_accuracy = 0.0

    for ep in range(epochs + 1):
        with tf.GradientTape() as t:
            logits = cora_gnn(fts, adj)
            loss = masked_softmax_cross_entropy(logits, labels, train_mask)
        variables = t.watched_variables()
        grad = t.gradient(loss, variables)
        optimizer.apply_gradients(zip(grad, variables))

        logits = cora_gnn(fts, adj)
        val_accuracy = masked_accuracy(logits, labels, val_mask)
        test_accuracy = masked_accuracy(logits, labels, test_mask)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            print(f'Epoch {ep} | Traninng Loss: {loss.numpy():.3f} | Val acc: {val_accuracy.numpy():.3f} Test Acc: {test_accuracy.numpy():.3f}| ')
    print('Completed')

In [50]:
print('Sum Pooling of neighbours')
train_cora(X, A, gnn)

Sum Pooling of neighbours
Epoch 0 | Traninng Loss: 2.020 | Val acc: 0.326 Test Acc: 0.310| 
Epoch 2 | Traninng Loss: 1.723 | Val acc: 0.540 Test Acc: 0.549| 
Epoch 3 | Traninng Loss: 1.391 | Val acc: 0.568 Test Acc: 0.603| 
Epoch 4 | Traninng Loss: 1.294 | Val acc: 0.610 Test Acc: 0.638| 
Epoch 5 | Traninng Loss: 1.146 | Val acc: 0.654 Test Acc: 0.662| 
Epoch 6 | Traninng Loss: 1.030 | Val acc: 0.708 Test Acc: 0.739| 
Epoch 7 | Traninng Loss: 0.927 | Val acc: 0.726 Test Acc: 0.751| 
Epoch 8 | Traninng Loss: 0.851 | Val acc: 0.728 Test Acc: 0.753| 
Epoch 9 | Traninng Loss: 0.784 | Val acc: 0.732 Test Acc: 0.746| 
Epoch 10 | Traninng Loss: 0.716 | Val acc: 0.736 Test Acc: 0.750| 
Epoch 11 | Traninng Loss: 0.653 | Val acc: 0.740 Test Acc: 0.744| 
Epoch 18 | Traninng Loss: 0.345 | Val acc: 0.744 Test Acc: 0.760| 
Epoch 24 | Traninng Loss: 0.207 | Val acc: 0.746 Test Acc: 0.764| 
Completed


In [64]:
# neighbours of neighbours

neighbours_of_neighbours_scalar=.1
A_neg= A+((A+A@A ==1) *1.0) * neighbours_of_neighbours_scalar 
A_neg = A_neg.astype('float32')

In [65]:
print('Sum Pooling of neighbours')
train_cora(X, A_neg, gnn)

Sum Pooling of neighbours
Epoch 0 | Traninng Loss: 2.047 | Val acc: 0.174 Test Acc: 0.164| 
Epoch 2 | Traninng Loss: 3.564 | Val acc: 0.232 Test Acc: 0.259| 
Epoch 3 | Traninng Loss: 2.360 | Val acc: 0.476 Test Acc: 0.469| 
Epoch 4 | Traninng Loss: 1.518 | Val acc: 0.534 Test Acc: 0.520| 
Epoch 5 | Traninng Loss: 1.552 | Val acc: 0.552 Test Acc: 0.503| 
Epoch 6 | Traninng Loss: 1.629 | Val acc: 0.618 Test Acc: 0.587| 
Epoch 7 | Traninng Loss: 1.428 | Val acc: 0.666 Test Acc: 0.632| 
Epoch 8 | Traninng Loss: 1.224 | Val acc: 0.708 Test Acc: 0.694| 
Epoch 9 | Traninng Loss: 1.088 | Val acc: 0.728 Test Acc: 0.740| 
Epoch 27 | Traninng Loss: 0.441 | Val acc: 0.734 Test Acc: 0.754| 
Epoch 28 | Traninng Loss: 0.421 | Val acc: 0.750 Test Acc: 0.767| 
Epoch 46 | Traninng Loss: 0.210 | Val acc: 0.752 Test Acc: 0.749| 
Epoch 48 | Traninng Loss: 0.194 | Val acc: 0.754 Test Acc: 0.752| 
Epoch 50 | Traninng Loss: 0.181 | Val acc: 0.758 Test Acc: 0.756| 
Epoch 60 | Traninng Loss: 0.126 | Val acc: 0.

In [66]:
# (2) Rely on yourself; Do not make use of your first neighbours: Node feature + Identity  matrix + GNN
print('Only itself')
train_cora(X, tf.eye(A.shape[0]), gnn)

Only itself
Epoch 0 | Traninng Loss: 1.946 | Val acc: 0.236 Test Acc: 0.294| 
Epoch 1 | Traninng Loss: 1.933 | Val acc: 0.360 Test Acc: 0.379| 
Epoch 2 | Traninng Loss: 1.914 | Val acc: 0.402 Test Acc: 0.431| 
Epoch 8 | Traninng Loss: 1.716 | Val acc: 0.418 Test Acc: 0.438| 
Epoch 9 | Traninng Loss: 1.672 | Val acc: 0.428 Test Acc: 0.452| 
Epoch 10 | Traninng Loss: 1.626 | Val acc: 0.442 Test Acc: 0.468| 
Epoch 11 | Traninng Loss: 1.577 | Val acc: 0.444 Test Acc: 0.480| 
Epoch 12 | Traninng Loss: 1.525 | Val acc: 0.446 Test Acc: 0.486| 
Epoch 16 | Traninng Loss: 1.297 | Val acc: 0.448 Test Acc: 0.488| 
Epoch 17 | Traninng Loss: 1.236 | Val acc: 0.450 Test Acc: 0.488| 
Epoch 19 | Traninng Loss: 1.111 | Val acc: 0.456 Test Acc: 0.495| 
Epoch 20 | Traninng Loss: 1.048 | Val acc: 0.458 Test Acc: 0.497| 
Epoch 21 | Traninng Loss: 0.984 | Val acc: 0.464 Test Acc: 0.498| 
Epoch 22 | Traninng Loss: 0.921 | Val acc: 0.470 Test Acc: 0.501| 
Epoch 23 | Traninng Loss: 0.859 | Val acc: 0.482 Test A

In [70]:
# (3) Make use of average of  your first neighbours (1) Node feature + Degree normalized ADJ matrix + GNN
print('Average Pooling')
degree_matrix = tf.reduce_sum(A, axis=-1)
train_cora(X, A / degree_matrix, gnn)

Average Pooling
Epoch 0 | Traninng Loss: 1.946 | Val acc: 0.124 Test Acc: 0.142| 
Epoch 2 | Traninng Loss: 1.918 | Val acc: 0.132 Test Acc: 0.151| 
Epoch 3 | Traninng Loss: 1.900 | Val acc: 0.134 Test Acc: 0.160| 
Epoch 4 | Traninng Loss: 1.880 | Val acc: 0.136 Test Acc: 0.161| 
Epoch 5 | Traninng Loss: 1.858 | Val acc: 0.144 Test Acc: 0.171| 
Epoch 6 | Traninng Loss: 1.835 | Val acc: 0.166 Test Acc: 0.188| 
Epoch 7 | Traninng Loss: 1.811 | Val acc: 0.208 Test Acc: 0.218| 
Epoch 8 | Traninng Loss: 1.784 | Val acc: 0.240 Test Acc: 0.248| 
Epoch 9 | Traninng Loss: 1.757 | Val acc: 0.268 Test Acc: 0.284| 
Epoch 10 | Traninng Loss: 1.727 | Val acc: 0.292 Test Acc: 0.326| 
Epoch 11 | Traninng Loss: 1.696 | Val acc: 0.324 Test Acc: 0.365| 
Epoch 12 | Traninng Loss: 1.663 | Val acc: 0.376 Test Acc: 0.392| 
Epoch 13 | Traninng Loss: 1.629 | Val acc: 0.426 Test Acc: 0.428| 
Epoch 14 | Traninng Loss: 1.592 | Val acc: 0.488 Test Acc: 0.474| 
Epoch 15 | Traninng Loss: 1.554 | Val acc: 0.528 Test A

In [73]:
# neighbours of neighbours

neighbours_of_neighbours_scalar=1.0
A_neg= A+((A+A@A ==1) *1.0) * neighbours_of_neighbours_scalar 
A_neg = A_neg.astype('float32')

degree_matrix_A_neg = tf.reduce_sum(A_neg, axis=-1)
train_cora(X, A_neg / degree_matrix_A_neg, gnn)

Epoch 0 | Traninng Loss: 1.946 | Val acc: 0.388 Test Acc: 0.389| 
Epoch 6 | Traninng Loss: 1.868 | Val acc: 0.404 Test Acc: 0.429| 
Epoch 7 | Traninng Loss: 1.852 | Val acc: 0.472 Test Acc: 0.475| 
Epoch 8 | Traninng Loss: 1.834 | Val acc: 0.530 Test Acc: 0.525| 
Epoch 9 | Traninng Loss: 1.814 | Val acc: 0.564 Test Acc: 0.563| 
Epoch 10 | Traninng Loss: 1.793 | Val acc: 0.596 Test Acc: 0.604| 
Epoch 11 | Traninng Loss: 1.771 | Val acc: 0.622 Test Acc: 0.630| 
Epoch 12 | Traninng Loss: 1.749 | Val acc: 0.634 Test Acc: 0.653| 
Epoch 13 | Traninng Loss: 1.725 | Val acc: 0.656 Test Acc: 0.673| 
Epoch 14 | Traninng Loss: 1.699 | Val acc: 0.688 Test Acc: 0.695| 
Epoch 15 | Traninng Loss: 1.671 | Val acc: 0.702 Test Acc: 0.712| 
Epoch 16 | Traninng Loss: 1.643 | Val acc: 0.718 Test Acc: 0.726| 
Epoch 17 | Traninng Loss: 1.613 | Val acc: 0.728 Test Acc: 0.741| 
Epoch 18 | Traninng Loss: 1.582 | Val acc: 0.740 Test Acc: 0.751| 
Epoch 19 | Traninng Loss: 1.551 | Val acc: 0.746 Test Acc: 0.748| 


In [74]:

# (4) It is a good idea to normalize adjesonsy matrix
norm_degr = tf.linalg.diag(1.0 / tf.sqrt(degree_matrix))
norm_A = tf.linalg.matmul(norm_degr, tf.matmul(A, norm_degr))
print('Normalized Pooling')
train_cora(X, norm_A, gnn)

Normalized Pooling
Epoch 0 | Traninng Loss: 1.946 | Val acc: 0.170 Test Acc: 0.201| 
Epoch 1 | Traninng Loss: 1.938 | Val acc: 0.208 Test Acc: 0.223| 
Epoch 2 | Traninng Loss: 1.928 | Val acc: 0.212 Test Acc: 0.232| 
Epoch 3 | Traninng Loss: 1.914 | Val acc: 0.220 Test Acc: 0.260| 
Epoch 4 | Traninng Loss: 1.899 | Val acc: 0.276 Test Acc: 0.305| 
Epoch 5 | Traninng Loss: 1.882 | Val acc: 0.368 Test Acc: 0.388| 
Epoch 6 | Traninng Loss: 1.864 | Val acc: 0.450 Test Acc: 0.464| 
Epoch 7 | Traninng Loss: 1.845 | Val acc: 0.500 Test Acc: 0.511| 
Epoch 8 | Traninng Loss: 1.825 | Val acc: 0.522 Test Acc: 0.546| 
Epoch 9 | Traninng Loss: 1.803 | Val acc: 0.558 Test Acc: 0.561| 
Epoch 10 | Traninng Loss: 1.779 | Val acc: 0.572 Test Acc: 0.589| 
Epoch 11 | Traninng Loss: 1.752 | Val acc: 0.586 Test Acc: 0.616| 
Epoch 12 | Traninng Loss: 1.724 | Val acc: 0.602 Test Acc: 0.636| 
Epoch 13 | Traninng Loss: 1.695 | Val acc: 0.610 Test Acc: 0.643| 
Epoch 14 | Traninng Loss: 1.664 | Val acc: 0.612 Test

## Convolutional Graph Neural Network in Detail


Let 
+ $X \in \mathbb{R}^{N \times d}$ be a matrix of node feature vectors.
+ $A \in \mathbb{R}^{N \times N}$ be an adjacency matrix
+ $D$ be a degree matrix ,i.e., $D_{ii}= \sum_j A_{ij}$

$$ H^{l+1} = \sigma \,( \tilde{D}^{-\frac{1}{2}} \, \tilde{A} \, \tilde{D}^{-\frac{1}{2}} \, H^l \, W^l)$$


+ $\tilde{A} = A + I$ is the adjacency matrix of the undirected graph G with added self-connections,i.e., $I$ is the indendity matrix.


+ $\tilde{D}_{ii}=\sum_j \tilde{A}_{ij}$


+ $H^l \in \mathbb{R}^{N \times D}$ is the matrix in the l.th layer. Hence $H^0$= X is a matrix of node feature vectors.


$$ 
A=\begin{bmatrix} 
0 \quad 1 \quad 0 \quad 0 \quad 0 \\
1 \quad 0 \quad 1 \quad 0 \quad 0 \\
0 \quad 1 \quad 0 \quad 1 \quad 1 \\
0 \quad 0 \quad 1 \quad 0 \quad 0 \\
0 \quad 0 \quad 1 \quad 0 \quad 0 \\
\end{bmatrix} 
$$
$$
D=\begin{bmatrix} 
1 \quad 0 \quad 0 \quad 0 \quad 0 \\
0 \quad 2 \quad 0 \quad 0 \quad 0 \\
0 \quad 0 \quad 3 \quad 0 \quad 0 \\
0 \quad 0 \quad 0 \quad 1 \quad 0 \\
0 \quad 0 \quad 0 \quad 0 \quad 1 \\
\end{bmatrix} 
$$ and $D^{-1}$ denotes the inverse of $D$.
$$
D^{-1}=\begin{bmatrix} 
1.0 \quad 0 \quad 0 \quad 0 \quad 0 \\
0 \quad .5 \quad 0 \quad 0 \quad 0 \\
0 \quad 0 \quad .3 \quad 0 \quad 0 \\
0 \quad 0 \quad 0 \quad 1. \quad 0 \\
0 \quad 0 \quad 0 \quad 0 \quad 1. \\
\end{bmatrix} 
$$

$$ D^{-1} \cdot A \cdot x=y,$$

$$ 
\begin{bmatrix} 
1.0 \quad 0 \quad 0 \quad 0 \quad 0 \\
0 \quad .5 \quad 0 \quad 0 \quad 0 \\
0 \quad 0 \quad .3 \quad 0 \quad 0 \\
0 \quad 0 \quad 0 \quad 1. \quad 0 \\
0 \quad 0 \quad 0 \quad 0 \quad 1. \\
\end{bmatrix} 
\cdot
\begin{bmatrix} 
0 \quad 1 \quad 0 \quad 0 \quad 0 \\
1 \quad 0 \quad 1 \quad 0 \quad 0 \\
0 \quad 1 \quad 0 \quad 1 \quad 1 \\
0 \quad 0 \quad 1 \quad 0 \quad 0 \\
0 \quad 0 \quad 1 \quad 0 \quad 0 \\
\end{bmatrix} 
\cdot
\begin{bmatrix} 1 \\
2 \\
3 \\
4 \\
5 \\
\end{bmatrix}
=
\begin{bmatrix} 
2. \\
2. \\
3.3 \\
3.0 \\
3.0 \\
\end{bmatrix}
$$

+ The vector representation of the $i-$th node is represented with $x_i$.
+ $A_{i:} \cdot x=y_i$ update the vector representation of the $i-$th node.
+ This means that $y_i$ is a sum of the features of the neighbouring nodes.


In [ ]:
import numpy as np
from scipy.linalg import sqrtm 

import networkx as nx
from scipy.linalg import sqrtm 
import matplotlib.pyplot as plt
from matplotlib import animation
%matplotlib inline
from IPython.display import HTML

# (1) A is given
A = np.array(
    [[0, 1, 0, 0, 0], 
     [1, 0, 1, 0, 0], 
     [0, 1, 0, 1, 1], 
     [0, 0, 1, 0, 0], 
     [0, 0, 1, 0, 0]])

# (2) Compute A tilde.
A_tilda = A + np.identity(len(A))

# (3) Compute D and D tilde.
D = np.zeros(A.shape)
np.fill_diagonal(D, A.sum(axis=0))
D_tilda = D + np.identity(len(D))
D_tilda_inv_sqrt=sqrtm(np.linalg.inv(D_tilda))

$$ \tilde A = A + I$$
$$ \tilde D = D + I$$


$$ \hat A = \tilde D ^{-\frac{1}{2}} \tilde A \tilde D ^{-\frac{1}{2}}$$


In [ ]:
D_tilda_inv_sqrt @ A_tilda @ D_tilda_inv_sqrt 

$$\sigma \,( \tilde{D}^{-\frac{1}{2}} \, \tilde{A} \, \tilde{D}^{-\frac{1}{2}} \, X \, W)$$


In [ ]:
X=np.random.randn(5,5)
W=np.random.randn(5,1)

# New Representation of input.
D_tilda_inv_sqrt @ A_tilda @ D_tilda_inv_sqrt @ X @ W

# Visualisation

In [ ]:
# (1) Create nx Network
g = nx.from_numpy_array(A)
# (2) Compute tilda
A_mod = A + np.eye(g.number_of_nodes())

# D for A_mod:
D_mod = np.zeros_like(A_mod)
np.fill_diagonal(D_mod, A_mod.sum(axis=1).flatten())

# Inverse square root of D:
D_mod_invroot = np.linalg.inv(sqrtm(D_mod))

In [ ]:
node_labels = {i: i+1 for i in range(g.number_of_nodes())}
pos = nx.planar_layout(g)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
nx.draw(
    g, pos, with_labels=True, 
    labels=node_labels, 
    node_color='#83C167', edge_color='gray', node_size=1500, font_size=30, font_family='serif'
)

In [ ]:
A_hat = D_mod_invroot @ A_mod @ D_mod_invroot
A_hat

In [ ]:
H = np.zeros((g.number_of_nodes(), 1))
H

In [ ]:
H[0,0] = 1 # the "water drop"

In [ ]:
# 
iters = 20
results = [H.flatten()]
for i in range(iters):
    H = A_hat @ H
    results.append(H.flatten())
print(f"Initial signal input: {results[0]}")
print(f"Final signal output after running {iters} steps of message-passing:  {results[-1]}")

In [ ]:
results

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

kwargs = {'cmap': 'hot', 'node_size': 1500, 'edge_color': 'gray', 
          'vmin': np.array(results).min(), 'vmax': np.array(results).max()*1.1}

def update(idx):
    ax.clear()
    colors = results[idx]
    nx.draw(g, pos, node_color=colors, ax=ax, **kwargs)
    ax.set_title(f"Iter={idx}", fontsize=20)

anim = animation.FuncAnimation(fig, update, frames=len(results), interval=1000, repeat=True)


In [ ]:
anim.save('water_drop_example.mp4')

In [ ]:
HTML(anim.to_html5_video())

# Graph Convolution Application

In [ ]:
from scipy.special import softmax
from networkx.algorithms.community.modularity_max import greedy_modularity_communities

In [ ]:
def draw_kkl(nx_G, label_map, node_color, pos=None, **kwargs):
    fig, ax = plt.subplots(figsize=(10,10))
    if pos is None:
        pos = nx.spring_layout(nx_G, k=5/np.sqrt(nx_G.number_of_nodes()))

    nx.draw(
        nx_G, pos, with_labels=label_map is not None, 
        labels=label_map, 
        node_color=node_color, 
        ax=ax, **kwargs)

In [ ]:

g = nx.karate_club_graph()
g.number_of_nodes(), g.number_of_edges()
communities = greedy_modularity_communities(g)
colors = np.zeros(g.number_of_nodes())
for i, com in enumerate(communities):
    colors[list(com)] = i

n_classes = np.unique(colors).shape[0]
labels = np.eye(n_classes)[colors.astype(int)]

club_labels = nx.get_node_attributes(g,'club')

_ = draw_kkl(g, None, colors, cmap='gist_rainbow', edge_color='gray')

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
pos = nx.spring_layout(g, k=5/np.sqrt(g.number_of_nodes()))
kwargs = {"cmap": 'gist_rainbow', "edge_color":'gray'}
nx.draw(
    g, pos, with_labels=False, 
    node_color=colors, 
    ax=ax, **kwargs)
#plt.savefig('karate_club_graph.png', bbox_inches='tight', transparent=True)

In [ ]:
# (1) Compute adjacency matrix
A = nx.to_numpy_matrix(g)
A

In [ ]:
# (2) Add self connections
A_mod = A + np.eye(g.number_of_nodes())

# (3) Compute D
D_mod = np.zeros_like(A_mod)
np.fill_diagonal(D_mod, np.asarray(A_mod.sum(axis=1)).flatten())

D_mod_invroot = np.linalg.inv(sqrtm(D_mod))

A_hat = D_mod_invroot @ A_mod @ D_mod_invroot

In [ ]:
X = np.eye(g.number_of_nodes())

In [ ]:
def glorot_init(nin, nout):
    sd = np.sqrt(6.0 / (nin + nout))
    return np.random.uniform(-sd, sd, size=(nin, nout))


def xent(pred, labels):
    return -np.log(pred)[np.arange(pred.shape[0]), np.argmax(labels, axis=1)]


def norm_diff(dW, dW_approx):
    return np.linalg.norm(dW - dW_approx) / (np.linalg.norm(dW) + np.linalg.norm(dW_approx))


class GradDescentOptim():
    def __init__(self, lr, wd):
        self.lr = lr
        self.wd = wd
        self._y_pred = None
        self._y_true = None
        self._out = None
        self.bs = None
        self.train_nodes = None
        
    def __call__(self, y_pred, y_true, train_nodes=None):
        self.y_pred = y_pred
        self.y_true = y_true
        
        if train_nodes is None:
            self.train_nodes = np.arange(y_pred.shape[0])
        else:
            self.train_nodes = train_nodes
            
        self.bs = self.train_nodes.shape[0]
        
    @property
    def out(self):
        return self._out
    
    @out.setter
    def out(self, y):
        self._out = y
    

class GCNLayer():
    def __init__(self, n_inputs, n_outputs, activation=None, name=''):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.W = glorot_init(self.n_outputs, self.n_inputs)
        self.activation = activation
        self.name = name
        
    def __repr__(self):
        return f"GCN: W{'_'+self.name if self.name else ''} ({self.n_inputs}, {self.n_outputs})"
        
    def forward(self, A, X, W=None):
        """
        Assumes A is (bs, bs) adjacency matrix and X is (bs, D), 
            where bs = "batch size" and D = input feature length
        """
        self._X = (A @ X).T # for calculating gradients.  (D, bs)
        
        if W is None:
            W = self.W
        
        H = W @ self._X # (h, D)*(D, bs) -> (h, bs)
        if self.activation is not None:
            H = self.activation(H)
        self._H = H # (h, bs)
        return self._H.T # (bs, h)
    
    def backward(self, optim, update=True):
        dtanh = 1 - np.asarray(self._H.T)**2 # (bs, out_dim)
        d2 = np.multiply(optim.out, dtanh)  # (bs, out_dim) *element_wise* (bs, out_dim)
        
        optim.out = d2 @ self.W # (bs, out_dim)*(out_dim, in_dim) = (bs, in_dim)
        
        dW = np.asarray(d2.T @ self._X.T) / optim.bs  # (out_dim, bs)*(bs, D) -> (out_dim, D)
        dW_wd = self.W * optim.wd / optim.bs # weight decay update
        
        if update:
            self.W -= (dW + dW_wd) * optim.lr 
        
        return dW + dW_wd

    
class SoftmaxLayer():
    def __init__(self, n_inputs, n_outputs, name=''):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.W = glorot_init(self.n_outputs, self.n_inputs)
        self.b = np.zeros((self.n_outputs, 1))
        self.name = name
        self._X = None # Used to calculate gradients
        
    def __repr__(self):
        return f"Softmax: W{'_'+self.name if self.name else ''} ({self.n_inputs}, {self.n_outputs})"
    
    def shift(self, proj):
        shiftx = proj - np.max(proj, axis=0, keepdims=True)
        exps = np.exp(shiftx)
        return exps / np.sum(exps, axis=0, keepdims=True)
        
    def forward(self, X, W=None, b=None):
        """Compute the softmax of vector x in a numerically stable way.
        
        X is assumed to be (bs, h)
        """
        self._X = X.T
        if W is None:
            W = self.W
        if b is None:
            b = self.b

        proj = np.asarray(W @ self._X) + b # (out, h)*(h, bs) = (out, bs)
        return self.shift(proj).T # (bs, out)
    
    def backward(self, optim, update=True):
        # should take in optimizer, update its own parameters and update the optimizer's "out"
        # Build mask on loss
        train_mask = np.zeros(optim.y_pred.shape[0])
        train_mask[optim.train_nodes] = 1
        train_mask = train_mask.reshape((-1, 1))
        
        # derivative of loss w.r.t. activation (pre-softmax)
        d1 = np.asarray((optim.y_pred - optim.y_true)) # (bs, out_dim)
        d1 = np.multiply(d1, train_mask) # (bs, out_dim) with loss of non-train nodes set to zero
        
        optim.out = d1 @ self.W # (bs, out_dim)*(out_dim, in_dim) = (bs, in_dim)
        
        dW = (d1.T @ self._X.T) / optim.bs  # (out_dim, bs)*(bs, in_dim) -> (out_dim, in_dim)
        db = d1.T.sum(axis=1, keepdims=True) / optim.bs # (out_dim, 1)
                
        dW_wd = self.W * optim.wd / optim.bs # weight decay update
        
        if update:   
            self.W -= (dW + dW_wd) * optim.lr
            self.b -= db.reshape(self.b.shape) * optim.lr
        
        return dW + dW_wd, db.reshape(self.b.shape)

In [ ]:
gcn1 = GCNLayer(g.number_of_nodes(), 2, activation=np.tanh, name='1')
sm1 = SoftmaxLayer(2, n_classes, "SM")
opt = GradDescentOptim(lr=0, wd=1.)
gcn1_out = gcn1.forward(A_hat, X)
opt(sm1.forward(gcn1_out), labels)

In [ ]:
def get_grads(inputs, layer, argname, labels, eps=1e-4, wd=0):
    cp = getattr(layer, argname).copy()
    cp_flat = np.asarray(cp).flatten()
    grads = np.zeros_like(cp_flat)
    n_parms = cp_flat.shape[0]
    for i, theta in enumerate(cp_flat):
        #print(f"Parm {argname}_{i}")
        theta_cp = theta
        
        # J(theta + eps)
        cp_flat[i] = theta + eps
        cp_tmp = cp_flat.reshape(cp.shape)
        predp = layer.forward(*inputs, **{argname: cp_tmp})
        wd_term = wd/2*(cp_flat**2).sum() / labels.shape[0]
        #print(wd_term)
        Jp = xent(predp, labels).mean() + wd_term
        
        # J(theta - eps)
        cp_flat[i] = theta - eps
        cp_tmp = cp_flat.reshape(cp.shape)
        predm = layer.forward(*inputs, **{argname: cp_tmp})
        wd_term = wd/2*(cp_flat**2).sum() / labels.shape[0]
        #print(wd_term)
        Jm = xent(predm, labels).mean() + wd_term
        
        # grad
        grads[i] = ((Jp - Jm) / (2*eps))
        
        # Back to normal
        cp_flat[i] = theta

    return grads.reshape(cp.shape)

In [ ]:
dW_approx = get_grads((gcn1_out,), sm1, "W", labels, eps=1e-4, wd=opt.wd)
db_approx = get_grads((gcn1_out,), sm1, "b", labels, eps=1e-4, wd=opt.wd)

In [ ]:
dW, db = sm1.backward(opt, update=False)

In [ ]:
assert norm_diff(dW, dW_approx) < 1e-7
assert norm_diff(db, db_approx) < 1e-7

In [ ]:

def get_gcn_grads(inputs, gcn, sm_layer, labels, eps=1e-4, wd=0):
    cp = gcn.W.copy()
    cp_flat = np.asarray(cp).flatten()
    grads = np.zeros_like(cp_flat)
    n_parms = cp_flat.shape[0]
    for i, theta in enumerate(cp_flat):
        theta_cp = theta
        
        # J(theta + eps)
        cp_flat[i] = theta + eps
        cp_tmp = cp_flat.reshape(cp.shape)
        pred = sm_layer.forward(gcn.forward(*inputs, W=cp_tmp))
        w2 = (cp_flat**2).sum()+(sm_layer.W.flatten()**2).sum()
        Jp = xent(pred, labels).mean() + wd/(2*labels.shape[0])*w2
        
        # J(theta - eps)
        cp_flat[i] = theta - eps
        cp_tmp = cp_flat.reshape(cp.shape)
        pred = sm_layer.forward(gcn.forward(*inputs, W=cp_tmp))
        w2 = (cp_flat**2).sum()+(sm_layer.W.flatten()**2).sum()
        Jm = xent(pred, labels).mean() + wd/(2*labels.shape[0])*w2
        
        # grad
        grads[i] = ((Jp - Jm) / (2*eps))
        
        # Back to normal
        cp_flat[i] = theta

    return grads.reshape(cp.shape)

In [ ]:
dW2 = gcn1.backward(opt, update=False)

In [ ]:
dW2_approx = get_gcn_grads((A_hat, X), gcn1, sm1, labels, eps=1e-4, wd=opt.wd)

In [ ]:
assert norm_diff(dW2, dW2_approx) < 1e-7

In [ ]:

class GCNNetwork():
    def __init__(self, n_inputs, n_outputs, n_layers, hidden_sizes, activation, seed=0):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.n_layers = n_layers
        self.hidden_sizes = hidden_sizes
        self.activation = activation
        
        np.random.seed(seed)
        
        self.layers = list()
        # Input layer
        gcn_in = GCNLayer(n_inputs, hidden_sizes[0], activation, name='in')
        self.layers.append(gcn_in)
        
        # Hidden layers
        for layer in range(n_layers):
            gcn = GCNLayer(self.layers[-1].W.shape[0], hidden_sizes[layer], activation, name=f'h{layer}')
            self.layers.append(gcn)
            
        # Output layer
        sm_out = SoftmaxLayer(hidden_sizes[-1], n_outputs, name='sm')
        self.layers.append(sm_out)
        
    def __repr__(self):
        return '\n'.join([str(l) for l in self.layers])
    
    def embedding(self, A, X):
        # Loop through all GCN layers
        H = X
        for layer in self.layers[:-1]:
            H = layer.forward(A, H)
        return np.asarray(H)
    
    def forward(self, A, X):
        # GCN layers
        H = self.embedding(A, X)
        
        # Softmax
        p = self.layers[-1].forward(H)
        
        return np.asarray(p)

In [ ]:

gcn_model = GCNNetwork(
    n_inputs=g.number_of_nodes(), 
    n_outputs=n_classes, 
    n_layers=2,
    hidden_sizes=[16, 2], 
    activation=np.tanh,
    seed=100,
)
gcn_model

In [ ]:
y_pred = gcn_model.forward(A_hat, X)
embed = gcn_model.embedding(A_hat, X)
xent(y_pred, labels).mean()

In [ ]:
pos = {i: embed[i,:] for i in range(embed.shape[0])}
_ = draw_kkl(g, None, colors, pos=pos, cmap='gist_rainbow', edge_color='gray')

In [ ]:
train_nodes = np.array([0, 1, 8])
test_nodes = np.array([i for i in range(labels.shape[0]) if i not in train_nodes])
opt2 = GradDescentOptim(lr=2e-2, wd=2.5e-2)

In [ ]:

embeds = list()
accs = list()
train_losses = list()
test_losses = list()

loss_min = 1e6
es_iters = 0
es_steps = 50
# lr_rate_ramp = 0 #-0.05
# lr_ramp_steps = 1000

for epoch in range(15000):
    
    y_pred = gcn_model.forward(A_hat, X)

    opt2(y_pred, labels, train_nodes)
    
#     if ((epoch+1) % lr_ramp_steps) == 0:
#         opt2.lr *= 1+lr_rate_ramp
#         print(f"LR set to {opt2.lr:.4f}")

    for layer in reversed(gcn_model.layers):
        layer.backward(opt2, update=True)
        
    embeds.append(gcn_model.embedding(A_hat, X))
    # Accuracy for non-training nodes
    acc = (np.argmax(y_pred, axis=1) == np.argmax(labels, axis=1))[
        [i for i in range(labels.shape[0]) if i not in train_nodes]
    ]
    accs.append(acc.mean())
    
    loss = xent(y_pred, labels)
    loss_train = loss[train_nodes].mean()
    loss_test = loss[test_nodes].mean()
    
    train_losses.append(loss_train)
    test_losses.append(loss_test)
    
    if loss_test < loss_min:
        loss_min = loss_test
        es_iters = 0
    else:
        es_iters += 1
        
    if es_iters > es_steps:
        print("Early stopping!")
        break
    
    if epoch % 100 == 0:
        print(f"Epoch: {epoch+1}, Train Loss: {loss_train:.3f}, Test Loss: {loss_test:.3f}")
        
train_losses = np.array(train_losses)
test_losses = np.array(test_losses)

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.log10(train_losses), label='Train')
ax.plot(np.log10(test_losses), label='Test')
ax.legend()
ax.grid()

In [ ]:
pos = {i: embeds[-1][i,:] for i in range(embeds[-1].shape[0])}
_ = draw_kkl(g, None, colors, pos=pos, cmap='gist_rainbow', edge_color='gray')

In [ ]:
fig, ax = plt.subplots()
_ = ax.plot(accs, marker='o')
ax.grid()
_ = ax.set(ylim=[0,1])

In [ ]:
N = 500
snapshots = np.linspace(0, len(embeds)-1, N).astype(int)

In [ ]:
# Build plot
fig, ax = plt.subplots(figsize=(10, 10))
kwargs = {'cmap': 'gist_rainbow', 'edge_color': 'gray', }#'node_size': 55}

def update(idx):
    ax.clear()
    embed = embeds[snapshots[idx]]
    pos = {i: embed[i,:] for i in range(embed.shape[0])}
    nx.draw(g, pos, node_color=colors, ax=ax, **kwargs)

anim = animation.FuncAnimation(fig, update, frames=snapshots.shape[0], interval=10, repeat=False)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
anim.save('embed_anim.mp4', dpi=300)